# Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from google.colab import drive

drive.mount('/content/drive')

# TODO: Fill this according to your drive loc
# DAVID: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'DeepSDF/'
# STANLEY: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ROB 498 3D Robot Perception/3D-RP-Project/DeepSDF/'
# DEEPAK: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'DeepSDF/'
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ROB 498 3D Robot Perception/3D-RP-Project/DeepSDF/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['data', 'requirements.txt', 'utils', 'params.json', 'reconstruction.py', 'hello_testing.py', '__pycache__', 'PCN.ipynb', 'Reconstruct.ipynb', 'Grasp Generation.ipynb', 'dataset.py', '.ipynb_checkpoints', 'config_files', 'model', 'results', 'scripts', 'obj_paths.pkl', 'Dataset.ipynb', 'eval']


If you did everything right, you should see something like this:
```
['DeepSDF.ipynb', 'data', 'models', 'requirements.txt', 'utils', 'params.json', 'Reconstruct.ipynb', 'Dataset.ipynb', 'hello_testing.py', 'model.py', '__pycache__']
```

<font color='red'>NOTE:</font> You don't have to clone the shared drive.
1. Create a shortcut of the shared folder (DeepSDF) and place in your drive.
2. Then change the directory on the notebook according to your gdrive.

In [ ]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

import time, os
os.environ["TZ"] = "US/Eastern"
time.tzset()

from hello_testing import hello
hello()

dataset_path = os.path.join(GOOGLE_DRIVE_PATH, 'dataset.py')
dataset_edit_time = time.ctime(os.path.getmtime(dataset_path))
print('dataset.py last edited on %s' % dataset_edit_time)

Working!
dataset.py last edited on Tue Nov 21 21:22:54 2023


Navigate to :

```drive > .... > 3D-RP-Project > dataset.py```

Double-click on the file to open it on the side.

## Remaining in reconstruction.py

In [ ]:
!pip install trimesh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 688.5/688.5 kB 6.3 MB/s eta 0:00:00


In [ ]:
import torch
import os
import model.model_sdf as sdf_model
from utils import utils_deepsdf
import trimesh
from results import runs_sdf
import results
import numpy as np
import config_files
import yaml

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def read_params(cfg):
    """Read the settings from the settings.yaml file. These are the settings used during training."""
    training_settings_path = os.path.join(os.path.dirname(runs_sdf.__file__),  cfg['folder_sdf'], 'settings.yaml')
    with open(training_settings_path, 'rb') as f:
        training_settings = yaml.load(f, Loader=yaml.FullLoader)

    return training_settings


def reconstruct_object(cfg, latent_code, obj_idx, model, coords_batches, grad_size_axis):
    """
    Reconstruct the object from the latent code and save the mesh.
    Meshes are stored as .obj files under the same folder cerated during training, for example:
    - runs_sdf/<datetime>/meshes_training/mesh_0.obj
    """
    sdf = utils_deepsdf.predict_sdf(latent_code, coords_batches, model)
    try:
        vertices, faces = utils_deepsdf.extract_mesh(grad_size_axis, sdf)
    except:
        print('Mesh extraction failed')
        return

    # save mesh as obj
    mesh_dir = os.path.join(os.path.dirname(runs_sdf.__file__), cfg['folder_sdf'], 'meshes_training')
    if not os.path.exists(mesh_dir):
        os.mkdir(mesh_dir)
    obj_path = os.path.join(mesh_dir, f"mesh_{obj_idx}.obj")
    trimesh.exchange.export.export_mesh(trimesh.Trimesh(vertices, faces), obj_path, file_type='obj')

In [ ]:
def main(cfg):
    training_settings = read_params(cfg)

    # Load the model
    weights = os.path.join(os.path.dirname(runs_sdf.__file__), cfg['folder_sdf'], 'weights.pt')

    model = sdf_model.SDFModel(
        num_layers=training_settings['num_layers'],
        skip_connections=training_settings['latent_size'],
        latent_size=training_settings['latent_size'],
        inner_dim=training_settings['inner_dim']).to(device)
    model.load_state_dict(torch.load(weights, map_location=device))

    # Extract mesh obtained with the latent code optimised at inference
    coords, grad_size_axis = utils_deepsdf.get_volume_coords(cfg['resolution'])
    coords = coords.to(device)

    # Split coords into batches because of memory limitations
    coords_batches = torch.split(coords, 100000)

    # Load paths
    str2int_path = os.path.join(os.path.dirname(results.__file__), 'idx_str2int_dict.npy')
    results_dict_path = os.path.join(os.path.dirname(runs_sdf.__file__), cfg['folder_sdf'], 'results.npy')

    # Load dictionaries
    str2int_dict = np.load(str2int_path, allow_pickle=True).item()
    #print(str2int_dict)
    results_dict = np.load(results_dict_path, allow_pickle=True).item()

    for obj_id_path in cfg['obj_ids']:
        # Get object index in the results dictionary
        obj_idx = str2int_dict[obj_id_path]  # index in collected latent vector
        # Get the latent code optimised during training
        latent_code = results_dict['best_latent_codes'][obj_idx]
        latent_code = torch.tensor(latent_code).to(device)

        reconstruct_object(cfg, latent_code, obj_idx, model, coords_batches, grad_size_axis)


if __name__ == '__main__':

    cfg_path = os.path.join(os.path.dirname(config_files.__file__), 'reconstruct_from_latent.yaml')
    with open(cfg_path, 'rb') as f:
        cfg = yaml.load(f, Loader=yaml.FullLoader)

    main(cfg)

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
